# 03 - 跨域迁移（统一接口：BatteryDataset + experiments）

本 notebook 使用 `src/experiments.py` 的统一实验接口，支持：

- 三种输入：`features` / `sequence` / `image`
- 单目标：`soh` 或 `rul`
- 双目标：`both`（SOH + RUL）
- 训练策略：`random_truncate=True`（模拟在线截断） + `random_eol_threshold=True`（动态RUL阈值）
- 动态阈值约束：阈值被限制在 **[0.65, 0.9]**，同时不低于电池最小SOH、不高于当前SOH
- `image` 的语义已固化为：**从循环0开始到当前cycle_idx（包含当前）** 的全部历史区间，并在循环轴上插值到固定 `window_size`（输出尺寸恒定）

注意：当前优先保证单数据集按电池划分 train/val/test 可跑通；跨数据集严格 zero-shot 后续再做。


In [ ]:
import sys
sys.path.insert(0, '..')

from src.utils import set_seed, get_device
set_seed(42)
device = get_device()
print('device:', device)

In [ ]:
from src.experiments import ExperimentConfig, run_experiments, print_results_table

## 1) 单数据集基线（建议先跑通）

先选一个数据集（例如 MATR），用 `features/sequence/image` 各跑一次，确认流程通畅。


In [ ]:
DATASETS = ['MATR']

config = ExperimentConfig(
    name='single_domain_image',
    datasets=DATASETS,
    input_type='image',
    models=['cnn2d'],
    target_type='both',
    epochs=5,
    batch_size=8,
    num_samples=200,
    window_size=100,
    lr=1e-3,
    weight_decay=1e-5,
    random_truncate=True,
    truncate_min_ratio=0.2,
    truncate_max_ratio=1.0,
    random_eol_threshold=True,
    eol_threshold_margin=0.01,
    default_eol_threshold=0.8,
    soh_loss_weight=1.0,
    rul_loss_weight=0.1,
    output_dir='results',
)
config.device = device
config

In [ ]:
results = run_experiments(config, processed_dir='../data/processed')
print_results_table(results)

## 2) 对比不同输入与模型（可选）

将 `input_type` 改为 `features` 或 `sequence`，或将 `models` 改为对应的兼容模型即可。

- `features` 兼容：`mlp`, `pinn`
- `sequence` 兼容：`cnn1d`, `lstm`, `bilstm`, `transformer`, `fno`
- `image` 兼容：`cnn2d`, `vit`, `simplevit`


In [ ]:
# 示例：时序输入 + LSTM
config2 = ExperimentConfig(
    name='single_domain_sequence',
    datasets=['MATR'],
    input_type='sequence',
    models=['lstm'],
    target_type='both',
    epochs=5,
    batch_size=16,
    num_samples=200,
    window_size=100,
    lr=1e-3,
    weight_decay=1e-5,
    random_truncate=True,
    random_eol_threshold=True,
    output_dir='results',
)
config2.device = device
results2 = run_experiments(config2, processed_dir='../data/processed')
print_results_table(results2)